In [9]:
#%matplotlib inline

In [2]:
files = {'train':'/common/home/vk405/Projects/Crossmdl/Data/Recipe1M features/embeddings_train1.pkl',
'valid':'/common/home/vk405/Projects/Crossmdl/Data/Recipe1M features/embeddings_val1.pkl',
'test':'/common/home/vk405/Projects/Crossmdl/Data/Recipe1M features/embeddings_test1.pkl'}

In [3]:
import pickle


with open(files['train'], 'rb') as f:
    data = pickle.load(f)

In [15]:
%%time
from cca_zoo.models import CCA
model = CCA(latent_dims=10)
model.fit((data[0],data[1]))


CPU times: user 38min 22s, sys: 15min 17s, total: 53min 40s
Wall time: 1min 26s


CCA(latent_dims=10, random_state=RandomState(MT19937) at 0x7F9B503B9640)

In [16]:
out = model.transform((data[0],data[1]))

In [17]:
out[0].shape

(281598, 10)

In [4]:
%%time
from sklearn.cross_decomposition import CCA
model = CCA(n_components=10)
model.fit(data[0],data[1])

In [ ]:
out2 = model.transform(data[0],data[1])

In [2]:
import os
os.listdir()

['embeddings_train1.pkl', 'embeddings_test1.pkl', 'embeddings_val1.pkl']

In [1]:
#!pip install pytorch_lightning

In [2]:
#!pip install cca-zoo[deep]


# Deep CCA

This example demonstrates how to easily train Deep CCA models and variants


In [6]:
import numpy as np
import pytorch_lightning as pl
from matplotlib import pyplot as plt
from torch.utils.data import Subset,Dataset

from cca_zoo.deepmodels import (
    DCCA,
    CCALightning,
    get_dataloaders,
    architectures,
    DCCA_NOI,
    DCCA_SDL,
    BarlowTwins,
)

In [7]:
class MyDataSet(Dataset):
    def __init__(self,rdata):
        self.rdata = rdata
    def __len__(self):
        return len(self.rdata)
    def __getitem__(self,idx):
        return self.rdata[idx]
   

In [9]:
v1 = np.random.randn(20000,1024)
v2 = np.random.randn(20000,1024)

In [ ]:
n_train = 15000
n_val = 5000
val_dataset = Subset(train_dataset, np.arange(n_train, n_train + n_val))
train_dataset = Subset(train_dataset, np.arange(n_train))

In [12]:
# The number of latent dimensions across models
latent_dims = 2
# number of epochs for deep models
epochs = 50

encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=1024)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=1024)
dcca = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

In [21]:
from cca_zoo.models import CCA
import numpy as np
rng=np.random.RandomState(0)
X1 = rng.random((100000,1024))
X2 = rng.random((100000,1024))
model = CCA(latent_dims=1)


In [22]:
model.fit((X1,X2)).score((X1,X2))

array([0.20063175])

In [ ]:

n_train = 500
n_val = 100
train_dataset = SplitMNISTDataset(root="",mnist_type="MNIST", train=True, download=True)
val_dataset = Subset(train_dataset, np.arange(n_train, n_train + n_val))
train_dataset = Subset(train_dataset, np.arange(n_train))
train_loader, val_loader = get_dataloaders(train_dataset, val_dataset, batch_size=128)


In [4]:
#from multiviewdata.torchdatasets import SplitMNISTDataset



def plot_latent_label(model, dataloader, num_batches=100):
    fig, ax = plt.subplots(ncols=model.latent_dims)
    for j in range(model.latent_dims):
        ax[j].set_title(f"Dimension {j}")
        ax[j].set_xlabel("View 1")
        ax[j].set_ylabel("View 2")
    for i, batch in enumerate(dataloader):
        z = model(*batch["views"])
        zx, zy = z
        zx = zx.to("cpu").detach().numpy()
        zy = zy.to("cpu").detach().numpy()
        for j in range(model.latent_dims):
            ax[j].scatter(zx[:, j], zy[:, j], c=batch["label"].numpy(), cmap="tab10")
        if i > num_batches:
            plt.colorbar()
            break




ModuleNotFoundError: No module named 'multiviewdata'

Deep CCA



In [ ]:
dcca = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])
dcca = CCALightning(dcca)
trainer = pl.Trainer(
    max_epochs=epochs,
    enable_checkpointing=False,
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
)
trainer.fit(dcca, train_loader, val_loader)
plot_latent_label(dcca.model, train_loader)
plt.suptitle("DCCA")
plt.show()

Deep CCA by Non-Linear Orthogonal Iterations



In [ ]:
dcca_noi = DCCA_NOI(
    latent_dims=latent_dims, N=len(train_dataset), encoders=[encoder_1, encoder_2]
)
dcca_noi = CCALightning(dcca_noi)
trainer = pl.Trainer(
    max_epochs=epochs,
    enable_checkpointing=False,
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
)
trainer.fit(dcca_noi, train_loader, val_loader)
plot_latent_label(dcca_noi.model, train_loader)
plt.suptitle("DCCA by Non-Linear Orthogonal Iterations")
plt.show()

Deep CCA by Stochastic Decorrelation Loss



In [ ]:
dcca_sdl = DCCA_SDL(
    latent_dims=latent_dims, N=len(train_dataset), encoders=[encoder_1, encoder_2]
)
dcca_sdl = CCALightning(dcca_sdl)
trainer = pl.Trainer(
    max_epochs=epochs,
    enable_checkpointing=False,
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
)
trainer.fit(dcca_sdl, train_loader, val_loader)
plot_latent_label(dcca_sdl.model, train_loader)
plt.suptitle("DCCA by Stochastic Decorrelation")
plt.show()

Deep CCA by Barlow Twins



In [ ]:
barlowtwins = BarlowTwins(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])
barlowtwins = CCALightning(barlowtwins)
trainer = pl.Trainer(
    max_epochs=epochs,
    enable_checkpointing=False,
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
)
trainer.fit(barlowtwins, train_loader, val_loader)
plot_latent_label(barlowtwins.model, train_loader)
plt.suptitle("DCCA by Barlow Twins")
plt.show()